Dataset: Student study hours, attendance, previous grades → final grade (0-100)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

x = torch.rand(1000, 4)
y = torch.rand(1000, 1)*100


x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1)

x_train = x_train.to('mps')
x_val = x_val.to('mps')
y_train = y_train.to('mps')
y_val = y_val.to('mps')

data_train = TensorDataset(x_train, y_train)
data_val = TensorDataset(x_val, y_val)


data_train = DataLoader(data_train, batch_size=64, shuffle=True)
data_val = DataLoader(data_val, batch_size=128)


class StudentGrader(nn.Module):
    def __init__(self):
        super(StudentGrader, self).__init__()
        self.f1 = nn.Linear(4, 128)
        self.f2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.f1(x))
        x = self.relu(self.f2(x))
        return self.output(x)

    
model = StudentGrader()
model = model.to('mps')
crit = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr=0.002)
model.train()
epochs = 10

for i in range(epochs):
    t_loss = 0
    for xi, yi in data_train: 
        optim.zero_grad()
        pred = model(xi)
        loss = crit(pred, yi)
        loss.backward()
        optim.step()
        t_loss += loss.item() 

print("training_loss", t_loss/len(data_train))


model.eval()
with torch.no_grad():
    loss = 0
    for xi, yi in data_val:
        pred = model(xi)
        loss += crit(pred, yi)

print("loss", loss/len(data_val))








training_loss -41282.0609375
loss tensor(-49169.9492, device='mps:0')
